In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# `Logit` on Orders - A warm-up challenge (~1h)

## Select features

🎯 Let's figure out the impact of `wait_time` and `delay_vs_expected` on very `good/bad reviews`

👉 Using our `orders` training_set, we will run two `multivariate logistic regressions`:
- `logit_one` to predict `dim_is_one_star` 
- `logit_five` to predict `dim_is_five_star`.

 

In [5]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

👉 Import your dataset:

In [6]:
from olist.order import Order
orders = Order().get_training_data(with_distance_seller_customer=True)

👉 Select in a list which features you want to use:

⚠️ Make sure you are not creating data leakage (i.e. selecting features that are derived from the target)

💡 To figure out the impact of `wait_time` and `delay_vs_expected` we need to control for the impact of other features, include in your list all features that may be relevant

In [11]:
orders.head()

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status,dim_is_five_star,dim_is_one_star,review_score,number_of_items,number_of_sellers,price,freight_value,distance_seller_customer
0,e481f51cbdc54678b7cc49136f2d6af7,8.436574,15.544063,0.0,delivered,0,0,4,1,1,29.99,8.72,18.063837
1,53cdb2fc8bc7dce0b6741e2150273451,13.782037,19.137766,0.0,delivered,0,0,4,1,1,118.70,22.76,856.292580
2,47770eb9100c2d0c44946d9cf07ec65d,9.394213,26.639711,0.0,delivered,1,0,5,1,1,159.90,19.22,514.130333
3,949d5b44dbf5de918fe9c16f97b45f8a,13.208750,26.188819,0.0,delivered,1,0,5,1,1,45.00,27.20,1822.800366
4,ad21c59c0840e6cb83a9ceb5573f8159,2.873877,12.112049,0.0,delivered,1,0,5,1,1,19.90,8.72,30.174037


In [12]:
orders_c = orders.copy()

In [26]:
logit_one = smf.logit(formula="dim_is_one_star ~ wait_time + delay_vs_expected + number_of_items + number_of_sellers + price + freight_value + distance_seller_customer", data=orders_c).fit()
logit_one.summary()

Optimization terminated successfully.
         Current function value: 0.273582
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        dim_is_one_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95864
Method:                           MLE   Df Model:                            7
Date:                Thu, 07 Nov 2024   Pseudo R-squ.:                  0.1448
Time:                        09:26:20   Log-Likelihood:                -26229.
converged:                       True   LL-Null:                       -30669.
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -5.2724      0.070    -74.899      0.000      -5.410      -5.134
wait_time                    0.0753      0.002     42.173      0.000       0.072       0.079
delay_vs_expected            0.0548      0.004     13.801      0.000       0.047       0.063
number_of_items              0.4665      0.020     23.634      0.000       0.428       0.505
number_of_sellers            1.4382      0.064     22.642      0.000       1.314       1.563
price                        0.0002   5.44e-05      4.464      0.000       0.000       0.000
freight_value               -0.0010      0.001     -1.697      0.090      -0.002       0.000
distance_seller_customer    -0.0003   2.32e-05    -12.934      0.000      -0.000      -0.000
============================================================================================
"""

OBSERVACIONES:

#Intercepto: Probabilidad baja de recibir una reseña de una estrella cuando todas las demás variables son cero.

#wait_time: coeficiente positivo a medida que aumenta el tiempo de espera, la probabilidad de recibir una reseña de una estrella también aumenta. Es significativo por p-v

#delay_vs_expected:coeficiente positivo los retrasos aumentan la probabilidad de recibir calificaciones bajas. Es significativo por p-v

#number_of_items y number_of_sellers: Si aumentamos los items comprados puede aumentar la cantidad de sellers por compra, lo que aumenta la prob de recibir malas reseñas. Es significativo por p-v

#price: Pedidos más caros pueden estar sujetos a una mayor probabilidad de recibir una reseña baja. Es significativo por p-v

#freight_value: p-value mayor, no tiene significancia estadistica.

In [28]:
logit_five = smf.logit(formula="dim_is_five_star ~ wait_time + delay_vs_expected + number_of_items + number_of_sellers + price + freight_value + distance_seller_customer", data=orders_c).fit()
logit_five.summary()

Optimization terminated successfully.
         Current function value: 0.636830
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:       dim_is_five_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95864
Method:                           MLE   Df Model:                            7
Date:                Thu, 07 Nov 2024   Pseudo R-squ.:                 0.05806
Time:                        09:27:14   Log-Likelihood:                -61054.
converged:                       True   LL-Null:                       -64817.
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    2.4534      0.064     38.132      0.000       2.327       2.579
wait_time                   -0.0552      0.001    -44.693      0.000      -0.058      -0.053
delay_vs_expected           -0.0930      0.005    -18.414      0.000      -0.103      -0.083
number_of_items             -0.2548      0.016    -16.331      0.000      -0.285      -0.224
number_of_sellers           -1.1542      0.063    -18.219      0.000      -1.278      -1.030
price                     9.997e-05   3.67e-05      2.721      0.007    2.79e-05       0.000
freight_value                0.0002      0.000      0.590      0.555      -0.001       0.001
distance_seller_customer     0.0001    1.4e-05     10.509      0.000       0.000       0.000
============================================================================================
"""

OBSERVACIONES:

#Intercepto: Probabilidad recibir una reseña de cinco estrellas de forma general.

#wait_time: cuando aumenta el tiempo de espera, la probabilidad de recibir una reseña de cinco estrellas disminuye. Es significativo por p-v

#delay_vs_expected:los retrasos reducen la probabilidad de recibir una reseña de cinco estrellas. Es significativo por p-v

#number_of_items y number_of_sellers: Si aumentamos los items comprados puede aumentar la cantidad de sellers por compra, lo que disminuye la prob de recibir una reseña de 5 estrellas. Es significativo por p-v

#price: Pedidos más caros pueden estar sujetos a una mayor probabilidad de recibir una reseña de 5 estrellas. Es significativo por p-v

#freight_value: p-value mayor, no tiene significancia estadistica.

🕵🏻 Check the `multi-colinearity` of your features, using the `VIF index`.

* It shouldn't be too high (< 10 preferably) to ensure that we can trust the partial regression coefficents and their associated `p-values` 
* Do not forget to standardize your data ! 
    * A `VIF Analysis` is made by regressing a feature vs. the other features...
    * So you want to `remove the effect of scale` so that your features have an equal importance before running any linear regression!
    
    
📚 <a href="https://www.statisticshowto.com/variance-inflation-factor/">Statistics How To - Variance Inflation Factor</a>

📚  <a href="https://online.stat.psu.edu/stat462/node/180/">PennState - Detecting Multicollinearity Using Variance Inflation Factors</a>

⚖️ Standardizing:

In [30]:
orders_c_scaled = orders_c.copy()
features = ['wait_time', 'delay_vs_expected', 'number_of_items', 'number_of_sellers', 'price', 'freight_value', 'distance_seller_customer']

In [31]:
for feature in features:
    mu = orders_c[feature].mean()
    sigma = orders_c[feature].std()
    orders_c_scaled[feature] = (orders_c_scaled[feature] - mu) / sigma

orders_c_scaled[features].head()

,wait_time,delay_vs_expected,number_of_items,number_of_sellers,price,freight_value,distance_seller_customer
0,-0.431192,-0.161781,-0.264595,-0.112544,-0.513802,-0.652038,-0.979475
1,0.134174,-0.161781,-0.264595,-0.112544,-0.086640,0.000467,0.429743
2,-0.329907,-0.161781,-0.264595,-0.112544,0.111748,-0.164053,-0.145495
3,0.073540,-0.161781,-0.264595,-0.112544,-0.441525,0.206815,2.054621
4,-1.019535,-0.161781,-0.264595,-0.112544,-0.562388,-0.652038,-0.959115


👉 Run your VIF Analysis to analyze the potential multicolinearities:

In [39]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [41]:
orders_c_scaled_features = orders_c_scaled[features]
vif_data = pd.DataFrame()
vif_data["features"] = orders_c_scaled_features.columns

vif_values = []
for i in range(orders_c_scaled_features.shape[1]):
    vif_value = vif(orders_c_scaled_features.values, i)
    vif_values.append(vif_value)

vif_data["vif_index"] = vif_values
vif_data = round(vif_data.sort_values(by="vif_index", ascending=False), 2)
vif_data

,features,vif_index
0,wait_time,2.62
1,delay_vs_expected,2.21
5,freight_value,1.67
6,distance_seller_customer,1.44
2,number_of_items,1.37
4,price,1.21
3,number_of_sellers,1.09


## Logistic Regressions

👉 Fit two `Logistic Regression` models:
- `logit_one` to predict `dim_is_one_star` 
- `logit_five` to predict `dim_is_five_star`.

`Logit 1️⃣`

In [37]:
logit_one = smf.logit(formula="dim_is_one_star ~ wait_time + delay_vs_expected + number_of_items + number_of_sellers + price + freight_value + distance_seller_customer", data=orders_c_scaled).fit()
logit_one.summary()

Optimization terminated successfully.
         Current function value: 0.273582
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        dim_is_one_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95864
Method:                           MLE   Df Model:                            7
Date:                Thu, 07 Nov 2024   Pseudo R-squ.:                  0.1448
Time:                        10:00:53   Log-Likelihood:                -26229.
converged:                       True   LL-Null:                       -30669.
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -2.4676      0.013   -190.852      0.000      -2.493      -2.442
wait_time                    0.7124      0.017     42.173      0.000       0.679       0.746
delay_vs_expected            0.2553      0.019     13.801      0.000       0.219       0.292
number_of_items              0.2500      0.011     23.634      0.000       0.229       0.271
number_of_sellers            0.1778      0.008     22.642      0.000       0.162       0.193
price                        0.0505      0.011      4.464      0.000       0.028       0.073
freight_value               -0.0219      0.013     -1.697      0.090      -0.047       0.003
distance_seller_customer    -0.1785      0.014    -12.934      0.000      -0.206      -0.151
============================================================================================
"""

`Logit 5️⃣`

In [38]:
logit_five = smf.logit(formula="dim_is_five_star ~ wait_time + delay_vs_expected + number_of_items + number_of_sellers + price + freight_value + distance_seller_customer", data=orders_c_scaled).fit()
logit_five.summary()

Optimization terminated successfully.
         Current function value: 0.636830
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:       dim_is_five_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95864
Method:                           MLE   Df Model:                            7
Date:                Thu, 07 Nov 2024   Pseudo R-squ.:                 0.05806
Time:                        10:00:55   Log-Likelihood:                -61054.
converged:                       True   LL-Null:                       -64817.
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.3388      0.007     47.340      0.000       0.325       0.353
wait_time                   -0.5219      0.012    -44.693      0.000      -0.545      -0.499
delay_vs_expected           -0.4330      0.024    -18.414      0.000      -0.479      -0.387
number_of_items             -0.1365      0.008    -16.331      0.000      -0.153      -0.120
number_of_sellers           -0.1427      0.008    -18.219      0.000      -0.158      -0.127
price                        0.0208      0.008      2.721      0.007       0.006       0.036
freight_value                0.0053      0.009      0.590      0.555      -0.012       0.023
distance_seller_customer     0.0872      0.008     10.509      0.000       0.071       0.103
============================================================================================
"""

#wait_time y delay_vs_expected: Estas variables son consistentes en ambos modelos. Aumentan la probabilidad de una reseña de una estrella y disminuyen la probabilidad de una reseña de cinco estrellas, destacando la importancia de tiempos de entrega rápidos y cumplidos.

#Control de otras variables: Variables como number_of_items, price, y distance_seller_customer tienen diferentes efectos en la probabilidad de recibir reseñas extremas, lo cual nos da un panorama más 
completo sobre los factores que influyen en la satisfacción del cliente.

💡 It's time to analyse the results of these two logistic regressions:

- Interpret the partial coefficients in your own words.
- Check their statistical significances with `p-values`
- Do you notice any differences between `logit_one` and `logit_five` in terms of coefficient importances?

In [43]:
# Among the following sentences, store the ones that are true in the list below

a = "delay_vs_expected influences five_star ratings even more than one_star ratings"
b = "wait_time influences five_star ratings even more more than one_star"

your_answer = [a]

🧪 __Test your code__

In [44]:
from nbresult import ChallengeResult

result = ChallengeResult('logit',
    answers = your_answer
)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /root/.pyenv/versions/3.10.6/envs/lewagon/bin/python
cachedir: .pytest_cache
rootdir: /root/code/MonicaVenzor/04-Decision-Science/04-Logistic-Regression/data-logit/tests
plugins: anyio-3.6.2, asyncio-0.19.0
asyncio: mode=strict
collecting ... collected 1 item

test_logit.py::TestLogit::test_question PASSED                           [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/logit.pickle

git commit -m 'Completed logit step'

git push origin master



<details>
    <summary>- <i>Explanations and advanced concepts </i> -</summary>


> _All other thing being equal, the `delay factor` tends to increase the chances of getting stripped of the 5-star even more so than it affect the chances of 1-star reviews. Probably because 1-stars are really targeting bad products themselves, not bad deliveries_
    
❗️ However, to be totally rigorous, we have to be **more careful when comparing coefficients from two different models**, because **they might not be based on similar populations**!
    We have 2 sub-populations here: (people who gave 1-stars; and people who gave 5-stars) and they may exhibit intrinsically different behavior patterns. It may well be that "happy-people" (who tends to give 5-stars easily) are less sensitive as "grumpy-people" (who shoot 1-stars like Lucky-Luke), when it comes to "delay", or "price"...

</details>


🏁 Congratulations! 

💾 Don't forget to commit and push your `logit.ipynb` notebook !